In [1]:
from mayavi import mlab
from tvtk.tools import visual
from numpy import array

In [2]:
import tensorflow as tf
import numpy as np
import t3f
tf.set_random_seed(0)
np.random.seed(0)
import matplotlib.pyplot as plt
import metric_util as mt
import data_util as du
from t3f import shapes
from nilearn import image
from skimage.measure import compare_ssim as ssim
from tensorflow.python.util import nest
import copy
from nilearn import plotting
from t3f import ops
import mri_draw_utils as mrd
from t3f import initializers
from t3f import approximate
from scipy import optimize 
from scipy import special
from nilearn.image.resampling import coord_transform
from scipy import stats
from mayavi.mlab import *
from tvtk.pyface.movie_maker import MovieMaker


/usr/local/lib/python2.7/site-packages/matplotlib/__init__.py:1405: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


In [3]:
subject_scan_path = du.get_full_path_subject1()
print "Subject Path: " + str(subject_scan_path)
x_true_org = mt.read_image_abs_path(subject_scan_path)

Subject Path: /work/pl/sch/analysis/data/COBRE001/swaAMAYER+cobre01_63001+M87100944+20110309at135133+RSTpre_V01_R01+CM.nii


In [4]:
data = np.array(image.index_img(x_true_org, 0).get_data())

In [5]:
x, y, z = np.mgrid[-78:78+1:3, -111:75+1:3, -51:84+1:3]

In [6]:
def get_i_j_k(x1, y1, z1):
    result = np.round(coord_transform(x1, y1, z1, np.linalg.inv(x_true_org.affine)))
    result = result.astype(int)
    result = (result[0], result[1], result[2])
    result = np.asarray(result).T
    return result[0], result[1],result [2]

In [7]:
def spatial_array(a, data, target):
    for i in range(a.shape[0]):
        for j in range(a.shape[1]):
            for k in range(a.shape[2]):
                #print i,j,k
                c1, c2, c3 = get_i_j_k(i,j,k)
                target[i,j,k] = data[c1,c2,c3]
    return target
    
    

In [8]:
target_array = np.zeros_like(data)


In [9]:
def get_slice(index, data):
    if index == 'x':
        res = data[0:data.shape[0],:,:]
    elif index == 'y':
        res = data[:,0:data.shape[1],:]
    else:
        res = data[:,:,0:data.shape[2]]
    return res
    

In [10]:
x1 = np.linspace(0,data.shape[0],data.shape[0], endpoint=True, dtype=int)

In [11]:
y1 = np.linspace(0,data.shape[1],data.shape[1], endpoint=True, dtype=int)

In [12]:
z1 = np.linspace(0,data.shape[1],data.shape[2], endpoint=True, dtype=int)

In [13]:
xv, yv, zv = np.meshgrid(x1, y1, z1, indexing = 'ij', sparse=False)
x2v, y2v, z2v = np.meshgrid(x1, y1, z1, indexing = 'ij', sparse=False)

In [14]:
dfdx, dfdy, dfdz = np.gradient(data)

In [15]:
z1


array([ 0,  1,  2,  4,  5,  7,  8,  9, 11, 12, 14, 15, 16, 18, 19, 21, 22,
       23, 25, 26, 28, 29, 30, 32, 33, 35, 36, 37, 39, 40, 42, 43, 44, 46,
       47, 49, 50, 51, 53, 54, 56, 57, 58, 60, 61, 63])

In [16]:
data.shape

(53, 63, 46)

In [17]:
y1

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 63])

In [18]:
f = mlab.figure(12, fgcolor=(.0, .0, .0), bgcolor=(1.0, 1.0, 1.0))

/usr/local/lib/python2.7/site-packages/tvtk/array_handler.py:268: FutureWarning: Conversion of the second argument of issubdtype from `complex` to `np.complexfloating` is deprecated. In future, it will be treated as `np.complex128 == np.dtype(complex).type`.
  assert not numpy.issubdtype(z.dtype, complex), \


In [19]:
countour_sf = mlab.contour3d(xv, yv, zv, data, contours=7, opacity=0.5, colormap='hsv')

In [20]:
vectors = mlab.quiver3d(x2v, y2v, z2v, dfdx, dfdy, dfdz, mode='arrow',scale_mode='vector', mask_points=8, opacity=0.8, colormap='jet')
   

In [21]:
mlab.outline(countour_sf, color=(0.7, .7, .7))

In [22]:
#mlab.pipeline.scalar_cut_plane(countour_sf, opacity = 0.8)

In [23]:
image_plane = mlab.pipeline.image_plane_widget(countour_sf,
                            plane_orientation='z_axes',
                            slice_index=38,transparent=True, opacity = 0.8
                        )

In [24]:

image_source = image_plane.mlab_source

In [25]:
zz1 = np.linspace(data.shape[2],0, endpoint=True, dtype=int)

In [26]:
zz1

array([46, 45, 44, 43, 42, 41, 40, 39, 38, 37, 36, 35, 34, 33, 32, 31, 30,
       30, 29, 28, 27, 26, 25, 24, 23, 22, 21, 20, 19, 18, 17, 16, 15, 15,
       14, 13, 12, 11, 10,  9,  8,  7,  6,  5,  4,  3,  2,  1,  0,  0])

In [27]:
folder = '/apps/git/python/image_utils/src/movie'
movie_name = folder + '/' + str('miss_ratio60_') + str('_') + str('.avi')

In [28]:
import  moviepy.editor as mpy
import moviepy.video.io.ffmpeg_tools as vid

In [29]:
fig = mlab.gcf()


In [30]:
image_plane.scene.movie_maker.directory=folder


In [31]:
image_plane.scene.movie_maker.filename = str('miss_ratio60_%05d.png')

In [32]:
with image_plane.scene.movie_maker.record_movie() as mm:
    for i in zz1:
        image_plane.ipw.slice_index = i
        mm.animation_step()
    file_name = folder + '/' + str('miss_ratio60_') + str(i) + str('_') + str('.tiff')
    mm.scene.save_tiff(file_name)

In [33]:
target_folder = '/apps/git/python/image_utils/src/movie/movie013'

In [34]:
#text.property.font_size = 8

In [35]:
import glob

file_list = []
for filename in glob.iglob(target_folder+ '/*.png'):
     print filename
     file_list.append(filename)
    

/apps/git/python/image_utils/src/movie/movie013/miss_ratio60_00000.png
/apps/git/python/image_utils/src/movie/movie013/miss_ratio60_00001.png
/apps/git/python/image_utils/src/movie/movie013/miss_ratio60_00002.png
/apps/git/python/image_utils/src/movie/movie013/miss_ratio60_00003.png
/apps/git/python/image_utils/src/movie/movie013/miss_ratio60_00004.png
/apps/git/python/image_utils/src/movie/movie013/miss_ratio60_00005.png
/apps/git/python/image_utils/src/movie/movie013/miss_ratio60_00006.png
/apps/git/python/image_utils/src/movie/movie013/miss_ratio60_00007.png
/apps/git/python/image_utils/src/movie/movie013/miss_ratio60_00008.png
/apps/git/python/image_utils/src/movie/movie013/miss_ratio60_00009.png
/apps/git/python/image_utils/src/movie/movie013/miss_ratio60_00010.png
/apps/git/python/image_utils/src/movie/movie013/miss_ratio60_00011.png
/apps/git/python/image_utils/src/movie/movie013/miss_ratio60_00012.png
/apps/git/python/image_utils/src/movie/movie013/miss_ratio60_00013.png
/apps/

In [36]:
file_list.shape

AttributeError: 'list' object has no attribute 'shape'

In [ ]:
clip = mpy.ImageSequenceClip(file_list, fps=15)
clip.write_gif("volume.gif")

In [ ]:
@mlab.show
@mlab.animate(delay=250, ui=True)
def anim():
   
    while 1:
         for i in zz1:
            image_plane.ipw.slice_index = i
            yield


In [ ]:
anim()

In [ ]:
#mlab.show()